In [2]:
!pip install pandas_ta
!pip install ccxt
!pip install pandas
!pip install numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=b38e2d2ce2afc142b92a76aa1c776070111841a2b3e31addb557860a073ecd9b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10


In [6]:
import ccxt
import pandas as pd
import pandas_ta as ta
import numpy as np

# Initialize exchange
exchange = ccxt.kucoin({'rateLimit': 1200, 'enableRateLimit': True})

# Fetch historical data
def fetch_data(symbol, timeframe, limit=500):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Calculate indicators
def calculate_indicators(df):
    df['RSI'] = ta.rsi(df['close'], length=14)
    df['EMA20'] = ta.ema(df['close'], length=20)
    df['EMA50'] = ta.ema(df['close'], length=50)
    df['EMA200'] = ta.ema(df['close'], length=200)

    # Bollinger Bands
    bbands = ta.bbands(df['close'], length=20)
    df['Bollinger_Upper'] = bbands['BBU_20_2.0']
    df['Bollinger_Middle'] = bbands['BBM_20_2.0']
    df['Bollinger_Lower'] = bbands['BBL_20_2.0']

    # ATR
    df['ATR'] = ta.atr(df['high'], df['low'], df['close'], length=14)
    return df


# Analyze multi-timeframe data
def analyze_timeframes(symbol):
    data_1d = fetch_data(symbol, '1d')
    data_4h = fetch_data(symbol, '4h')
    data_15m = fetch_data(symbol, '15m')

    data_1d = calculate_indicators(data_1d)
    data_4h = calculate_indicators(data_4h)
    data_15m = calculate_indicators(data_15m)

    return data_1d, data_4h, data_15m

# Generate trading signals
def generate_signals(data_1d, data_4h, data_15m):
    signals = []

    # Evaluate trend alignment
    trend_1d = data_1d['EMA20'].iloc[-1] > data_1d['EMA50'].iloc[-1]
    trend_4h = data_4h['EMA20'].iloc[-1] > data_4h['EMA50'].iloc[-1]

    if trend_1d and trend_4h:
        # 15M chart entry condition
        latest = data_15m.iloc[-1]
        previous = data_15m.iloc[-2]

        # RSI divergence and Bollinger Band breakout
        if (latest['RSI'] < 30 and latest['close'] > latest['EMA20'] and
            previous['close'] < previous['Bollinger_Lower'] and
            latest['close'] > latest['Bollinger_Lower']):

            entry_price = latest['close']
            take_profit = entry_price + 2 * latest['ATR']
            stop_loss = entry_price - latest['ATR']

            signals.append({
                'symbol': symbol,
                'entry': entry_price,
                'take_profit': take_profit,
                'stop_loss': stop_loss
            })
    return signals

# Main execution
symbols = ["BTC/USDT", "ETH/USDT", "NEAR/USDT", "AVAX/USDT", "SOL/USDT"]

all_signals = []
for symbol in symbols:
    print(f"Analyzing {symbol}...")
    data_1d, data_4h, data_15m = analyze_timeframes(symbol)
    signals = generate_signals(data_1d, data_4h, data_15m)
    all_signals.extend(signals)

# Display signals
for signal in all_signals:
    print(f"Signal: {signal}")


Analyzing BTC/USDT...
Analyzing ETH/USDT...
Analyzing NEAR/USDT...
Analyzing AVAX/USDT...
Analyzing SOL/USDT...
